# This file will train a network on many replicas of a set, then produce the mean CFFs found

In [48]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import numpy as np
import pandas as pd 
from BHDVCS_torch import TBHDVCS

import matplotlib
import matplotlib.pyplot as plt

import sys
import time
from scipy.stats import chisquare

In [49]:
tb = TBHDVCS()

f = tb.TotalUUXS_curve_fit3
loss_func = tb.loss_MSE_errs

In [50]:
dats = pd.read_csv('dvcs_psuedo.csv')
n = np.array(dats['#Set'])
ind = np.array(dats['index'])
k = np.array(dats['k'])
qq = np.array(dats['QQ'])
xb = np.array(dats['x_b'])
t = np.array(dats['t'])
phi = np.array(dats['phi_x'])
F = np.array(dats['F'])
errF = np.array(dats['errF']) 
F1 = np.array(dats['F1'])
F2 = np.array(dats['F2'])
const = np.array(dats['dvcs'])  
ReH_target = np.array(dats['ReH']) 
ReE_target = np.array(dats['ReE']) 
ReHT_target = np.array(dats['ReHtilde'])
yrep = []

errs_H = []
errs_E = []
errs_HT = []

rep_ReH = []
rep_ReE = []
rep_ReHT = []

In [51]:
blank_net = torch.nn.Sequential(
        torch.nn.Linear(4, 100),
        torch.nn.Tanh(),
        torch.nn.Linear(100, 100),
        torch.nn.Tanh(),
        torch.nn.Linear(100, 80),
        torch.nn.Tanh(),
        torch.nn.Linear(80, 3)
    )

In [52]:
optimizer = torch.optim.Adam(blank_net.parameters(), lr=0.02)

In [53]:
EPOCH = 2500

In [54]:
datset = 1

In [55]:
reps = 100

## Training:

In [57]:
i = datset
a = 36*i # start index of set
b = a+36 # end index of set



for j in range(reps): # create n replicas
    start = time.time()
    net = blank_net # untrain/reset network
    
    yrep = [0] * (b-a) # create array to be filled with replicated F values
    
    for l in range(b-a): # populate yrep with random normal values with mean = F and sd = errF
        
        yind = a+l # index of data point 
        yrep[l] = (np.random.normal(F[yind], errF[yind]))

    
    xdat = np.array([phi[a:b], qq[a:b], xb[a:b], t[a:b], k[a:b], F1[a:b], F2[a:b], const[a:b]])
    ydat = np.array(yrep)

    x = Variable(torch.from_numpy(xdat[1:5].transpose()))
    y = Variable(torch.from_numpy(ydat.transpose()))

    xdat = Variable(torch.from_numpy(xdat))

    errs = Variable(torch.from_numpy(errF[a:b]))

    for epoch in range(EPOCH):

        p = net(x.float()) #output 3 predicted values for cffs

        ReHfit = torch.mean(torch.transpose(p, 0, 1)[0])
        ReEfit = torch.mean(torch.transpose(p, 0, 1)[1])
        ReHTfit = torch.mean(torch.transpose(p, 0, 1)[2])
        cffs = [ReHfit, ReEfit, ReHTfit]

        loss = loss_func((xdat.float()), cffs, errs, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        pctdone = ((float(2500*(j)+epoch)*100)/float(reps*EPOCH))
        sys.stdout.write("\r%.2f%% done with set #%d." % ((pctdone, i)))
        sys.stdout.flush()

    rep_ReH.append(cffs[0].data.numpy())
    rep_ReE.append(cffs[1].data.numpy())
    rep_ReHT.append(cffs[2].data.numpy())
    
    print("\nReH on replica %d: %.2f vs real ReH: %.2f" % (j, cffs[0], ReH_target[a]))
    print("ReE on replica %d: %.2f vs real ReE: %.2f" % (j, cffs[1], ReE_target[a]))
    print("ReHT on replica %d: %.2f vs real ReHT: %.2f" % (j, cffs[2], ReHT_target[a]))

rep_ReH = np.array(rep_ReH)
rep_ReE = np.array(rep_ReE)
rep_ReHT = np.array(rep_ReHT)

err_H = abs(100*(abs(np.mean(rep_ReH)-ReH_target[a]))/ReH_target[a])
err_E = abs(100*(abs(np.mean(rep_ReE)-ReE_target[a]))/ReE_target[a])
err_HT = abs(100*(abs(np.mean(rep_ReHT)-ReHT_target[a]))/ReHT_target[a])


print('\nMean ReH for set %d = %.2f, error = %.2f, variance = %.2f' % (i, np.mean(rep_ReH), err_H, np.var(rep_ReH)))
print('Mean ReE for set %d = %.2f, error = %.2f, variance = %.2f' % (i, np.mean(rep_ReE), err_E, np.var(rep_ReE)))
print('Mean ReHT for set %d = %.2f, error = %.2f, variance = %.2f\n' % (i, np.mean(rep_ReHT), err_HT, np.var(rep_ReHT)))

1.00% done with set #1.
ReH on replica 0: 12.95 vs real ReH: 12.55
ReE on replica 0: -53.91 vs real ReE: -52.55
ReHT on replica 0: 8.19 vs real ReHT: 6.97
2.00% done with set #1.
ReH on replica 1: 12.34 vs real ReH: 12.55
ReE on replica 1: -51.01 vs real ReE: -52.55
ReHT on replica 1: 8.87 vs real ReHT: 6.97
3.00% done with set #1.
ReH on replica 2: 10.01 vs real ReH: 12.55
ReE on replica 2: -38.74 vs real ReE: -52.55
ReHT on replica 2: 8.65 vs real ReHT: 6.97
4.00% done with set #1.
ReH on replica 3: 9.88 vs real ReH: 12.55
ReE on replica 3: -38.73 vs real ReE: -52.55
ReHT on replica 3: 7.85 vs real ReHT: 6.97
5.00% done with set #1.
ReH on replica 4: 8.38 vs real ReH: 12.55
ReE on replica 4: -30.43 vs real ReE: -52.55
ReHT on replica 4: 8.44 vs real ReHT: 6.97
6.00% done with set #1.
ReH on replica 5: 9.44 vs real ReH: 12.55
ReE on replica 5: -36.36 vs real ReE: -52.55
ReHT on replica 5: 7.08 vs real ReHT: 6.97
7.00% done with set #1.
ReH on replica 6: 15.42 vs real ReH: 12.55
ReE on

53.00% done with set #1.
ReH on replica 52: 12.35 vs real ReH: 12.55
ReE on replica 52: -51.64 vs real ReE: -52.55
ReHT on replica 52: 7.48 vs real ReHT: 6.97
54.00% done with set #1.
ReH on replica 53: 14.20 vs real ReH: 12.55
ReE on replica 53: -60.33 vs real ReE: -52.55
ReHT on replica 53: 5.62 vs real ReHT: 6.97
55.00% done with set #1.
ReH on replica 54: 12.22 vs real ReH: 12.55
ReE on replica 54: -50.31 vs real ReE: -52.55
ReHT on replica 54: 7.80 vs real ReHT: 6.97
56.00% done with set #1.
ReH on replica 55: 8.51 vs real ReH: 12.55
ReE on replica 55: -30.78 vs real ReE: -52.55
ReHT on replica 55: 8.75 vs real ReHT: 6.97
57.00% done with set #1.
ReH on replica 56: 13.37 vs real ReH: 12.55
ReE on replica 56: -55.87 vs real ReE: -52.55
ReHT on replica 56: 8.83 vs real ReHT: 6.97
58.00% done with set #1.
ReH on replica 57: 6.19 vs real ReH: 12.55
ReE on replica 57: -18.49 vs real ReE: -52.55
ReHT on replica 57: 9.21 vs real ReHT: 6.97
59.00% done with set #1.
ReH on replica 58: 14.6